# Web Scraping - Houses

In [3]:
# Importing libraries

from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm_notebook
from math import ceil
from datetime import datetime

In [4]:
def url_reader(url) -> object:

    """"
    This function reads the html code of the url and returns a BeautifulSoup object.

    Parameters:
    url (str): The url to be read.
    """

    # Need to set user agent to avoid 403 error
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = response.read().decode('utf-8')
        soup_obj = BeautifulSoup(html, 'html.parser')
        return soup_obj

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
        

In [5]:
def list_of_ads(soup) -> list:
    
    """
    This function returns a list of all the ads in the page.

    Parameters:
    soup (object): The BeautifulSoup object of the page.

    Returns:
    ad_links (list): A list of all the ads in the page.
    """

    ad_list = soup.findAll('a', class_='fnmrjs-0 fyjObc')

    ad_links = []
    for ad in ad_list:
        ad_links.append((ad.get('href')))
        
    return ad_links

In [6]:
def text_from_tag(tag):

    """
    This function returns the text from a tag.

    Parameters:
    tag (object): The BeautifulSoup object of the page.

    Returns:
    text (str): The text from the tag.
    """

    if tag is not None:
        return tag.get_text()
    else:
        return 'None'

In [7]:
def main(get_images=False, number_of_pages=None):

    """
    This function scrapes the olx website and returns a dataframe with all the ads properties.

    Parameters:
    get_images (bool): If True, the images of the ads will be downloaded.
    number_of_pages (int): The number of pages to be scraped.

    Returns:
    df (pandas.DataFrame): A dataframe with all the ads properties.
    """

    # Reads the main page
    main_page = 'https://al.olx.com.br/imoveis'
    soup = url_reader(main_page)

    # Gets the number of pages and ads
    aux = soup.find('span', class_='sc-1mi5vq6-0 eDXljX sc-ifAKCX fhJlIo').get_text().split()
    ad_per_page = int(aux[2])
    n_total_ads = int(aux[-2].replace('.', ''))


    # If number_of_pages is not specified, calculate the number of pages
    if number_of_pages is not None:
      n_pages = number_of_pages
    else:
      n_pages = ceil(n_total_ads/ad_per_page)

    # Initialize the dataframe
    all_ad_data = []

    # Loop over the pages
    for page_number in tqdm_notebook(range(n_pages), desc='Pages', colour='red'):
        soup = url_reader(main_page + '?o={}'.format(page_number+1))
        ad_links = list_of_ads(soup)
        
        # Loop through all the ads
        for ad in ad_links:
            soup = url_reader(ad)
            if soup is not None:
            
              # Title
              title_tag = soup.find('h1', class_='sc-45jt43-0 eCghYu sc-ifAKCX cmFKIN')
              title = text_from_tag(title_tag)

              # ID
              id_tag = soup.find('span', class_='sc-16iz3i7-0 qJvUT sc-ifAKCX fizSrB')
              id_ad = text_from_tag(id_tag).split()[-1]

              # Date and hour
              datetime_tag = soup.find('span', class_='sc-1oq8jzc-0 jvuXUB sc-ifAKCX fizSrB')
              datetime = text_from_tag(datetime_tag).split()
              date_ad, hour_ad = datetime[2], datetime[-1]

              # Description
              description_tag = soup.find('span', class_='sc-1sj3nln-1 eOSweo sc-ifAKCX cmFKIN')
              description = text_from_tag(description_tag).replace('\n',' ')

              # Price
              price_tag = soup.find('h2', class_='sc-ifAKCX eQLrcK')
              price = text_from_tag(price_tag).replace('.','')

              # Ad class
              calss_ad = 'amateur'
              if soup.find('span', class_='sc-16bj9n5-0 IIBHN sc-ifAKCX fizSrB') is not None: 
                  calss_ad = 'professional'

              # Product data
              data_tags = soup.findAll('div', class_='sc-hmzhuo sc-1f2ug0x-3 ONRJp sc-jTzLTM iwtnNi')
              data = []
              subdata = []

              for tag in data_tags:
                tag_soup = BeautifulSoup(str(tag), 'html.parser')
                data.append(tag_soup.find('dt', class_='sc-1f2ug0x-0 cLGFbW sc-ifAKCX cmFKIN').get_text())

                subdata_tag = tag_soup.find('a', class_='sc-57pm5w-0 sc-1f2ug0x-2 dBeEuJ')
                if subdata_tag == None:
                  subdata_tag = tag_soup.find('dd', class_ = 'sc-1f2ug0x-1 ljYeKO sc-ifAKCX kaNiaQ')

                subdata.append(subdata_tag.get_text())

              
              # Get images
              if get_images:
                  img_tags = soup.find('div', class_='h3us20-6 fAprjt')
                  
                  if img_tags is not None:
                    img_list = img_tags.findAll('img', class_='image')
                    if img_list is not None:
                      for i, img in enumerate(img_list):
                          urlretrieve(img.get('src'), './output/img/' + id_ad + '_' + str(i) + '.jpg')

              # Create a dictionary with all the data
              ad_data = {}
              ad_data['Title'] = title
              ad_data['ID'] = id_ad
              ad_data['Description'] = description
              ad_data['Date'] = date_ad
              ad_data['Hour'] = hour_ad
              ad_data['Class'] = calss_ad
              ad_data['Price'] = price

              for i, _ in enumerate(range(len(data))):
                  ad_data[data[i]] = subdata[i]

              ad_data['Link'] = ad

              # Append the dictionary to the list
              all_ad_data.append(ad_data)

            else:
              continue

    return all_ad_data

In [8]:
def create_csv(data):

    """
    This function creates a csv file with all the ads properties.

    Parameters:
    data (list): A list with all the ads properties.

    Returns:
    None
    """

    dataset = pd.DataFrame(data)

    today = datetime.now().strftime('%Y-%m-%d')

    return dataset.to_csv('./output/data/' + today + '_dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [9]:
# if number_of_pages is not specified, scrape all the ads from all the pages
alldata = main(get_images = True, number_of_pages=1)
create_csv(alldata)

Pages:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# creates a dataframe and shows the first 5 rows
dataset = pd.DataFrame(alldata)
dataset.head()

,Title,ID,Description,Date,Hour,Class,Price,Categoria,Tipo,Área útil,...,Município,Bairro,Logradouro,Link,Área construída,Tamanho,Condomínio,Acomoda,Características,IPTU
0,"90m² - 03 quartos, s/ 02stes, varanda, 02vgs.",772593301,Vendo excelente Apartamento completo de armári...,03/12,07:31,professional,400000,Apartamentos,Venda - apartamento padrão,90m²,...,Maceió,Jatiúca,Rua Doutor Roland Simon,https://al.olx.com.br/alagoas/imoveis/90m-03-q...,NaN,NaN,NaN,NaN,NaN,NaN
1,"Casa em Paripueira, 03quartos, s/ 01suite, pis...",903700303,"Oportunidade!!! Vendo casa em paripueira, com ...",03/12,07:31,professional,0,Casas,Venda - casa em rua pública,NaN,...,Paripueira,NaN,NaN,https://al.olx.com.br/alagoas/imoveis/casa-em-...,NaN,NaN,NaN,NaN,NaN,NaN
2,"Lançamento Qto e sala e dois quartos, em 120meses",648419387,Conheça o mais novo lançamento da V2 Construçõ...,03/12,07:29,professional,0,Apartamentos,Venda - apartamento padrão,39m²,...,Maceió,Ponta Verde,Avenida Professor Victal Barbosa,https://al.olx.com.br/alagoas/imoveis/lancamen...,NaN,NaN,NaN,NaN,NaN,NaN
3,Casa no condomínio De lá Roche (nova),891017443,Linda casa recém construída em um condomínio f...,03/12,07:29,amateur,850000,Casas,Venda - casa em condominio fechado,NaN,...,Arapiraca,Canafístula,Rua Rita Leão de Melo,https://al.olx.com.br/alagoas/imoveis/casa-no-...,212m²,NaN,NaN,NaN,NaN,NaN
4,"Flexeiras 12 hectares a 500 metros da BR, apen...",964654704,"Oportunidade, propriedade 12 hectares em Flexe...",03/12,07:20,professional,120000,"Terrenos, sítios e fazendas",Sítios e chácaras,NaN,...,Flexeiras,NaN,NaN,https://al.olx.com.br/alagoas/terrenos/flexeir...,NaN,120000m²,NaN,NaN,NaN,NaN
